# Optimización Bayesiana y Selección del Modelo Final

Este notebook realiza optimización avanzada de hiperparámetros con Optuna, validación robusta (CV anidada), ensembles y selección del modelo final para la predicción de consumo de sustancias.

## Tabla de Contenidos
- Caso 0 — Preparación e Importaciones
- Caso 1 — Carga de Datos y Verificaciones
- Caso 2 — Esquema de Validación Robusta (CV anidada)
- Caso 3 — Espacios de Hiperparámetros por Modelo
- Caso 4 — Optimización Multi‑Objetivo con Optuna (F1 + ROC‑AUC)
- Caso 5 — Validación Externa y Estabilidad de Métricas
- Caso 6 — Ensembles (Voting, Weighted Voting, Stacking) y Diversidad
- Caso 7 — Selección del Modelo Final y Artefactos para Producción
- Resumen de Decisiones Técnicas

Nota: TimeSeriesSplit no aplica aquí; los datos son transversales de encuesta (ENCSPA 2019). Se utiliza StratifiedKFold con semillas controladas para garantizar reproducibilidad y equilibrio por clase.

## Caso 0 — Preparación e Importaciones
Importamos librerías y fijamos semillas. Se incluyen `xgboost`, `lightgbm` y `randomforest` como modelos base, y soporte opcional para `catboost`. Para Optuna se usa TPE (búsqueda bayesiana) y un pruner mediano para acelerar convergencia.

In [9]:
# Imports y configuración
import pandas as pd
import numpy as np
import warnings, time, json
from pathlib import Path
import joblib
from datetime import datetime

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

import xgboost as xgb
import lightgbm as lgb

try:
    from catboost import CatBoostClassifier
    CATBOOST_AVAILABLE = True
except Exception:
    CATBOOST_AVAILABLE = False

warnings.filterwarnings('ignore')
SEED = 42
np.random.seed(SEED)
print("Librerías importadas y configuración lista")

Librerías importadas y configuración lista


## Caso 1 — Carga de Datos y Verificaciones
Se cargan los datos procesados del notebook 03 (`data/processed`). Si no están disponibles, se aplica un **pipeline básico** con imputación, encoding y escalado. Se conserva el balance de train (SMOTE) cuando existe para que el tuning refleje un conjunto equilibrado.

In [10]:
# Carga de datos procesados (del notebook 03) con fallback básico
print("Cargando datos procesados ...")
paths = Path('../data/processed')
use_smote = True

try:
    X_train_final = pd.read_pickle(paths/'X_train_balanced.pkl')
    y_train_final = pd.read_pickle(paths/'y_train_balanced.pkl')
    X_test_final = pd.read_pickle(paths/'X_test_transformed.pkl')
    y_test_final = pd.read_pickle(paths/'y_test.pkl')
    feature_names = pd.read_pickle(paths/'feature_names.pkl')
    print(f"Datos cargados: Train={X_train_final.shape}, Test={X_test_final.shape}, Features={len(feature_names)}")
except Exception as e:
    print(f"Fallo al cargar datos procesados: {e}")
    print("Usando pipeline básico como fallback (SIN SMOTE)")
    df = pd.read_csv('../data/g_capitulos.csv')
    target_var = 'G_11_F'
    categorical_features = ['G_01', 'G_02', 'G_03', 'G_04', 'G_05']
    numerical_features = ['G_06_A','G_06_B','G_06_C','G_06_D','G_07','G_08_A','G_08_B','G_01_A','G_02_A']
    df_clean = df[df[target_var].isin([1,2])].copy()
    X = df_clean[categorical_features + numerical_features]
    y = (df_clean[target_var] == 1).astype(int)
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import LabelEncoder, StandardScaler
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)
    cat_imputer = SimpleImputer(strategy='most_frequent')
    num_imputer = SimpleImputer(strategy='median')
    X_train[categorical_features] = cat_imputer.fit_transform(X_train[categorical_features])
    X_test[categorical_features] = cat_imputer.transform(X_test[categorical_features])
    X_train[numerical_features] = num_imputer.fit_transform(X_train[numerical_features])
    X_test[numerical_features] = num_imputer.transform(X_test[numerical_features])
    le_dict = {}
    for col in categorical_features:
        le = LabelEncoder()
        X_train[col] = le.fit_transform(X_train[col].astype(str))
        X_test[col] = le.transform(X_test[col].astype(str))
        le_dict[col] = le
    scaler = StandardScaler()
    X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
    X_test[numerical_features] = scaler.transform(X_test[numerical_features])
    X_train_final, y_train_final = X_train, y_train
    X_test_final, y_test_final = X_test, y_test
    feature_names = X_train_final.columns.tolist()
    use_smote = False

print("Verificación final:")
print(f"Train balance (positivos): {y_train_final.mean():.3f} | Test balance: {y_test_final.mean():.3f}")

Cargando datos procesados ...
Datos cargados: Train=(31442, 14), Test=(4686, 14), Features=14
Verificación final:
Train balance (positivos): 0.500 | Test balance: 0.161


## Caso 2 — Esquema de Validación Robusta (CV anidada)
Se usa validación cruzada **anidada**:
- `inner_cv` (3 folds): tuning de hiperparámetros y estimación de métricas promedio.
- `outer_cv` (5 folds): evaluación externa para medir **estabilidad** (media y desviación estándar).
Se reportan `F1` y `ROC‑AUC` con probabilidades cuando el modelo lo permite.

In [11]:
# Configuración de validación y métricas
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED+1)

def cv_metrics(estimator, X, y, cv):
    f1s, aucs = [], []
    for train_idx, val_idx in cv.split(X, y):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        est = estimator
        est.fit(X_tr, y_tr)
        y_pred = est.predict(X_val)
        f1s.append(f1_score(y_val, y_pred))
        if hasattr(est, 'predict_proba'):
            y_prob = est.predict_proba(X_val)[:,1]
            aucs.append(roc_auc_score(y_val, y_prob))
        else:
            aucs.append(np.nan)
    return float(np.nanmean(f1s)), float(np.nanmean(aucs)), f1s, aucs

print("Validación configurada: outer=5 folds, inner=3 folds")

Validación configurada: outer=5 folds, inner=3 folds


## Caso 3 — Espacios de Hiperparámetros por Modelo
Se definen espacios bayesianos con rangos razonables para cada modelo. La función `make_estimator` construye el estimador con parámetros sugeridos por Optuna.

In [12]:
# Espacios de hiperparámetros
def suggest_params(trial, model_name):
    if model_name == 'xgb':
        return {
            'n_estimators': trial.suggest_int('n_estimators', 150, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 8),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.5, 2.0),
            'min_child_weight': trial.suggest_float('min_child_weight', 1.0, 10.0)
        }
    elif model_name == 'lgb':
        return {
            'n_estimators': trial.suggest_int('n_estimators', 150, 500),
            'num_leaves': trial.suggest_int('num_leaves', 20, 120),
            'max_depth': trial.suggest_int('max_depth', 3, 8),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 100)
        }
    elif model_name == 'rf':
        max_feat = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
        return {
            'n_estimators': trial.suggest_int('n_estimators', 200, 600),
            'max_depth': trial.suggest_categorical('max_depth', [None, 10, 15, 20, 25, 30]),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'max_features': max_feat,
            'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
        }
    elif model_name == 'cat' and CATBOOST_AVAILABLE:
        return {
            'iterations': trial.suggest_int('iterations', 200, 600),
            'depth': trial.suggest_int('depth', 4, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0)
        }
    else:
        raise ValueError(f"Modelo no soportado en espacio: {model_name}")

def make_estimator(model_name, params):
    if model_name == 'xgb':
        return xgb.XGBClassifier(random_state=SEED, eval_metric='logloss', verbosity=0, n_jobs=-1, **params)
    elif model_name == 'lgb':
        return lgb.LGBMClassifier(random_state=SEED, verbose=-1, force_col_wise=True, **params)
    elif model_name == 'rf':
        return RandomForestClassifier(random_state=SEED, n_jobs=-1, **params)
    elif model_name == 'cat' and CATBOOST_AVAILABLE:
        return CatBoostClassifier(random_state=SEED, verbose=False, loss_function='Logloss', **params)
    else:
        raise ValueError(f"Modelo no soportado: {model_name}")

print("Espacios de hiperparámetros listos")

Espacios de hiperparámetros listos


## Caso 4 — Optimización Multi‑Objetivo con Optuna (F1 + ROC‑AUC)
Se crea un estudio con dos objetivos a maximizar: `F1` y `ROC‑AUC`. Se usa **TPE** con `MedianPruner` para detener configuraciones poco prometedoras. Se ejecutan **≥220 trials** por modelo.

In [13]:
# Optuna: estudio multi-objetivo (F1 y ROC-AUC) por modelo
def run_optuna_for_model(model_name, n_trials=220):
    sampler = TPESampler(seed=SEED)
    pruner = MedianPruner(n_startup_trials=30)
    study = optuna.create_study(directions=['maximize','maximize'], sampler=sampler, pruner=pruner)
    def objective(trial):
        params = suggest_params(trial, model_name)
        est = make_estimator(model_name, params)
        f1_mean, auc_mean, f1s, aucs = cv_metrics(est, X_train_final, y_train_final, inner_cv)
        trial.set_user_attr('f1s', f1s)
        trial.set_user_attr('aucs', aucs)
        trial.set_user_attr('params', params)
        return f1_mean, auc_mean
    start = time.time()
    study.optimize(objective, n_trials=n_trials, n_jobs=1)
    duration = time.time() - start
    print(f"Optuna {model_name}: {len(study.trials)} trials en {duration:.1f}s")
    return study

# Top 3 modelos del commit anterior (04): LightGBM, XGBoost, RandomForest
selected_models = ['lgb', 'xgb', 'rf']
studies = {}
for m in selected_models:
    studies[m] = run_optuna_for_model(m, n_trials=220)

print("Optimización completa para modelos seleccionados")

[I 2025-10-27 23:22:31,159] A new study created in memory with name: no-name-49c1a788-7569-4748-8141-50a2258dbfff
[I 2025-10-27 23:22:32,656] Trial 0 finished with values: [0.9239250026845865, 0.9770765355587431] and parameters: {'n_estimators': 281, 'num_leaves': 116, 'max_depth': 7, 'learning_rate': 0.07661100707771368, 'subsample': 0.6624074561769746, 'colsample_bytree': 0.662397808134481, 'reg_alpha': 0.05808361216819946, 'reg_lambda': 0.8661761457749352, 'min_child_samples': 64}.
[I 2025-10-27 23:22:33,981] Trial 1 finished with values: [0.92324989083568, 0.9764302482545807] and parameters: {'n_estimators': 398, 'num_leaves': 22, 'max_depth': 8, 'learning_rate': 0.16967533607196555, 'subsample': 0.6849356442713105, 'colsample_bytree': 0.6727299868828402, 'reg_alpha': 0.18340450985343382, 'reg_lambda': 0.3042422429595377, 'min_child_samples': 57}.
[I 2025-10-27 23:22:35,186] Trial 2 finished with values: [0.9155195249142046, 0.9686364062494098] and parameters: {'n_estimators': 301,

Optuna lgb: 220 trials en 333.0s


[I 2025-10-27 23:28:05,301] Trial 0 finished with values: [0.9217315629654874, 0.975777946518631] and parameters: {'n_estimators': 281, 'max_depth': 8, 'learning_rate': 0.1205712628744377, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.5871254182522991, 'min_child_weight': 8.795585311974417}.
[I 2025-10-27 23:28:06,611] Trial 1 finished with values: [0.914769371363887, 0.9671568514086654] and parameters: {'n_estimators': 360, 'max_depth': 7, 'learning_rate': 0.010725209743171996, 'subsample': 0.9879639408647978, 'colsample_bytree': 0.9329770563201687, 'reg_alpha': 0.21233911067827616, 'reg_lambda': 0.7727374508106509, 'min_child_weight': 2.650640588680904}.
[I 2025-10-27 23:28:07,398] Trial 2 finished with values: [0.9216615766984265, 0.9753644467985021] and parameters: {'n_estimators': 256, 'max_depth': 6, 'learning_rate': 0.04345454109729477, 'subsample': 0.7164916560792167, 'colsample_bytree': 0.844741157888

Optuna xgb: 220 trials en 204.9s


[I 2025-10-27 23:31:32,454] Trial 0 finished with values: [0.9088672126768363, 0.9651790306990836] and parameters: {'max_features': 'log2', 'n_estimators': 440, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 10, 'bootstrap': True}.
[I 2025-10-27 23:31:38,829] Trial 1 finished with values: [0.9112017869912478, 0.9614331105678704] and parameters: {'max_features': None, 'n_estimators': 410, 'max_depth': 15, 'min_samples_split': 10, 'min_samples_leaf': 8, 'bootstrap': False}.
[I 2025-10-27 23:31:43,029] Trial 2 finished with values: [0.9130833195251635, 0.9604432032500189] and parameters: {'max_features': None, 'n_estimators': 268, 'max_depth': 15, 'min_samples_split': 15, 'min_samples_leaf': 5, 'bootstrap': False}.
[I 2025-10-27 23:31:46,276] Trial 3 finished with values: [0.9149055328961214, 0.9681203682624252] and parameters: {'max_features': 'log2', 'n_estimators': 465, 'max_depth': 25, 'min_samples_split': 19, 'min_samples_leaf': 9, 'bootstrap': False}.
[I 2025-10-27 23:

Optuna rf: 220 trials en 855.9s
Optimización completa para modelos seleccionados


## Caso 5 — Validación Externa y Estabilidad de Métricas
Para cada modelo se elige el mejor `trial` y se evalúa con `outer_cv` (5 folds), reportando medias y desviaciones estándar de `F1` y `ROC‑AUC`. Esto permite analizar **robustez** y **variabilidad** del rendimiento.

In [14]:
# Selección del mejor trial y validación externa (outer CV)
def pick_best_trial(study):
    trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    trials_sorted = sorted(trials, key=lambda t: (t.values[0], t.values[1]), reverse=True)
    return trials_sorted[0]

best_models = {}
best_params = {}
cv_summary = {}

for m, st in studies.items():
    bt = pick_best_trial(st)
    params = bt.user_attrs['params']
    est = make_estimator(m, params)
    f1_mean, auc_mean, f1s, aucs = cv_metrics(est, X_train_final, y_train_final, outer_cv)
    best_models[m] = est.fit(X_train_final, y_train_final)
    best_params[m] = params
    cv_summary[m] = {
        'inner_best_f1': float(bt.values[0]),
        'inner_best_auc': float(bt.values[1]),
        'outer_f1_mean': float(np.nanmean(f1s)),
        'outer_f1_std': float(np.nanstd(f1s)),
        'outer_auc_mean': float(np.nanmean(aucs)),
        'outer_auc_std': float(np.nanstd(aucs)),
    }
    print(f"[{m}] inner_f1={bt.values[0]:.4f}, inner_auc={bt.values[1]:.4f} | outer_f1={cv_summary[m]['outer_f1_mean']:.4f}, outer_auc={cv_summary[m]['outer_auc_mean']:.4f}")

print("Validación externa (outer CV) completada")

[lgb] inner_f1=0.9258, inner_auc=0.9773 | outer_f1=0.9258, outer_auc=0.9775
[xgb] inner_f1=0.9243, inner_auc=0.9767 | outer_f1=0.9246, outer_auc=0.9767
[rf] inner_f1=0.9247, inner_auc=0.9731 | outer_f1=0.9247, outer_auc=0.9732
Validación externa (outer CV) completada


## Caso 6 — Ensembles (Voting, Weighted Voting, Stacking) y Diversidad
Se construyen ensembles para combinar fortalezas de los modelos:
- **Voting soft**: promedio de probabilidades.
- **Weighted soft voting**: ponderación por AUC externo.
- **Stacking**: meta-learner `LogisticRegression` sobre `predict_proba`.
Además, se calcula la **correlación** de probabilidades en test para medir diversidad (menor correlación sugiere mayor complementariedad).

In [15]:
# Evaluación en test y construcción de ensembles
def evaluate_on_test(name, model):
    y_pred = model.predict(X_test_final)
    f1 = f1_score(y_test_final, y_pred)
    if hasattr(model, 'predict_proba'):
        y_prob = model.predict_proba(X_test_final)[:,1]
        auc = roc_auc_score(y_test_final, y_prob)
    else:
        auc = np.nan
    acc = accuracy_score(y_test_final, y_pred)
    prec = precision_score(y_test_final, y_pred)
    rec = recall_score(y_test_final, y_pred)
    print(f"{name:20s} | F1={f1:.4f} AUC={auc:.4f} ACC={acc:.4f} PREC={prec:.4f} REC={rec:.4f}")
    return {'f1':float(f1), 'auc':float(auc), 'accuracy':float(acc), 'precision':float(prec), 'recall':float(rec)}

results_test = {}
for m, mdl in best_models.items():
    results_test[m] = evaluate_on_test(m, mdl)

# Soft Voting ensemble
estimators = [(m, best_models[m]) for m in selected_models]
voting_soft = VotingClassifier(estimators=estimators, voting='soft', n_jobs=-1)
voting_soft.fit(X_train_final, y_train_final)
results_test['voting_soft'] = evaluate_on_test('voting_soft', voting_soft)

# Weighted Soft Voting (pesos por outer AUC)
weights = [cv_summary[m]['outer_auc_mean'] for m in selected_models]
weights = [w / (sum(weights) + 1e-8) for w in weights]

class WeightedVotingProb:
    def __init__(self, models, weights):
        self.models = models
        self.weights = weights
    def fit(self, X, y):
        return self
    def predict_proba(self, X):
        probs = [mdl.predict_proba(X)[:,1] for mdl in self.models]
        prob = np.zeros(len(probs[0]))
        for p, w in zip(probs, self.weights):
            prob += w * p
        return np.vstack([1-prob, prob]).T
    def predict(self, X):
        return (self.predict_proba(X)[:,1] >= 0.5).astype(int)

weighted_soft = WeightedVotingProb([best_models[m] for m in selected_models], weights)
results_test['weighted_soft'] = evaluate_on_test('weighted_soft', weighted_soft)

# Stacking con meta-learner
stack = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(max_iter=1000, solver='saga', random_state=SEED),
    stack_method='predict_proba',
    passthrough=False,
    n_jobs=-1
)
stack.fit(X_train_final, y_train_final)
results_test['stacking'] = evaluate_on_test('stacking', stack)

# Diversidad entre modelos base (probabilidades en test)
def diversity(models, X):
    probs = {m: models[m].predict_proba(X)[:,1] for m in models}
    names = list(probs.keys())
    mat = np.zeros((len(names), len(names)))
    for i in range(len(names)):
        for j in range(len(names)):
            mat[i,j] = np.corrcoef(probs[names[i]], probs[names[j]])[0,1]
    return names, mat

names, corr_mat = diversity(best_models, X_test_final)
print('Correlación de probabilidades entre modelos base (test):')
print(pd.DataFrame(corr_mat, index=names, columns=names).round(3))

print("Evaluación en test y ensembles completada")

lgb                  | F1=0.6386 AUC=0.9225 ACC=0.8664 PREC=0.5660 REC=0.7325
xgb                  | F1=0.6534 AUC=0.9220 ACC=0.8698 PREC=0.5721 REC=0.7616
rf                   | F1=0.6617 AUC=0.9182 ACC=0.8745 PREC=0.5849 REC=0.7616
voting_soft          | F1=0.6537 AUC=0.9241 ACC=0.8709 PREC=0.5756 REC=0.7563
weighted_soft        | F1=0.6556 AUC=0.9241 ACC=0.8720 PREC=0.5785 REC=0.7563
stacking             | F1=0.6581 AUC=0.9216 ACC=0.8756 PREC=0.5905 REC=0.7430
Correlación de probabilidades entre modelos base (test):
       lgb    xgb     rf
lgb  1.000  0.997  0.972
xgb  0.997  1.000  0.972
rf   0.972  0.972  1.000
Evaluación en test y ensembles completada


## Caso 7 — Selección del Modelo Final y Artefactos para Producción
Se selecciona el modelo final priorizando `F1` y desempate por `ROC‑AUC`. Se consideran señales de **estabilidad** (desviación estándar en outer CV) y **robustez**. Se guardan el modelo y metadatos en `models/`.

In [16]:
# Selección del modelo final y guardado de artefactos
def pick_final_model(results_test, cv_summary):
    ranking = sorted(results_test.items(), key=lambda kv: (kv[1]['f1'], kv[1]['auc']), reverse=True)
    final_name = ranking[0][0]
    return final_name

final_name = pick_final_model(results_test, cv_summary)
if final_name in best_models:
    final_model = best_models[final_name]
elif final_name == 'voting_soft':
    final_model = voting_soft
elif final_name == 'weighted_soft':
    final_model = weighted_soft
elif final_name == 'stacking':
    final_model = stack
else:
    final_model = voting_soft

# Guardado de modelo y metadatos
models_dir = Path('../models')
models_dir.mkdir(parents=True, exist_ok=True)

model_path = models_dir / 'mejor_modelo_final.pkl'
joblib.dump(final_model, model_path)

metadata = {
    'fecha': datetime.utcnow().isoformat() + 'Z',
    'usa_smote': use_smote,
    'modelos_seleccionados_prev_commit': selected_models,
    'modelo_final': final_name,
    'resultados_test': results_test,
    'resumen_cv_outer': cv_summary,
    'hiperparametros': {m: best_params[m] for m in best_params},
    'weights_soft': [float(w) for w in weights],
    'diversidad_corr_test': pd.DataFrame(corr_mat, index=names, columns=names).round(6).to_dict(),
    'validacion': {
        'outer_cv': {'n_splits': 5, 'shuffle': True, 'random_state': SEED},
        'inner_cv': {'n_splits': 3, 'shuffle': True, 'random_state': SEED+1},
        'multi_objetivo': ['f1','roc_auc'],
        'optuna_trials_por_modelo': 220
    }
}
with open(models_dir / 'metadatos_modelo_final.json', 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print(f"Modelo final: {final_name} guardado en {model_path}")

Modelo final: rf guardado en ..\models\mejor_modelo_final.pkl


## Resumen de Decisiones Técnicas
- Validación: StratifiedKFold **anidado** (inner para tuning, outer para estabilidad).
- Optuna: **TPE** (búsqueda bayesiana) + `MedianPruner`, estudio **multi‑objetivo** (F1 y ROC‑AUC).
- Modelos: LightGBM, XGBoost, RandomForest (top 3 del commit 04).
- Ensembles: Voting soft, Voting ponderado por AUC, Stacking con `LogisticRegression`.
- Diversidad: correlación de probabilidades en test entre modelos base.
- Selección final: maximiza F1, desempata por AUC y considera **robustez** (std en outer CV).
- Artefactos: modelo y metadatos guardados en `models/`.